In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
df = pd.read_csv('data/food_crises_cleaned_with_market_details.csv', parse_dates=['date'])

### remove incorrect values in commodities

In [22]:
s_df = df[['date','district','ipc','category', 'commodity', 'usdprice']]

# remove incorrect columns and commodities that are null
s_df = s_df.drop(s_df[(s_df['commodity'] == 'Exchange rate') | (s_df['commodity'] == 'Exchange rate (unofficial)')].index)
s_df = s_df.dropna(subset=['commodity'])

# display final commodities in df
s_df.commodity.unique()

array(['Maize (white)', 'Sorghum (white, imported)',
       'Groundnuts (shelled)', 'Wheat flour', 'Beans (red)', 'Sesame',
       'Fuel (diesel)', 'Fuel (petrol-gasoline)', 'Oil (vegetable)',
       'Rice', 'Livestock (goat, medium-sized male)',
       'Livestock (sheep, medium-sized male)', 'Sugar (brown, imported)',
       'Fuel (diesel, parallel market)',
       'Wage (non-qualified labour, non-agricultural)',
       'Fuel (petrol-gasoline, parallel market)', 'Sorghum (local)',
       'Milling cost (sorghum)', 'Salt', 'Maize meal', 'Millet (white)',
       'Cassava (dry)', 'Cassava'], dtype=object)

In [23]:
s_df.columns

Index(['date', 'district', 'ipc', 'category', 'commodity', 'usdprice'], dtype='object')

In [24]:
s_df.head(20)

,date,district,ipc,category,commodity,usdprice
16,2008-05-01,Bor,NaN,cereals and tubers,Maize (white),1.3525
17,2008-05-01,Bor,NaN,cereals and tubers,"Sorghum (white, imported)",1.2881
18,2008-05-01,Bor,NaN,pulses and nuts,Groundnuts (shelled),2.8983
19,2008-06-01,Bor,NaN,cereals and tubers,Maize (white),1.3284
20,2008-06-01,Bor,NaN,cereals and tubers,"Sorghum (white, imported)",1.2881
21,2008-06-01,Bor,NaN,pulses and nuts,Groundnuts (shelled),2.8983
23,2008-08-01,Bor,NaN,cereals and tubers,Maize (white),1.9322
24,2008-08-01,Bor,NaN,cereals and tubers,"Sorghum (white, imported)",1.2881
25,2008-08-01,Bor,NaN,pulses and nuts,Groundnuts (shelled),2.6567
26,2008-09-01,Bor,NaN,cereals and tubers,Maize (white),2.0529


### get average price per date, district, and category

In [25]:
# Calculate the average usdprice per date, district, and category
result_df = s_df.groupby(['date', 'district', 'category'])['usdprice'].mean().reset_index()

# Merge the result_df with s_df on 'date', 'district', and 'category'
s_df = s_df.merge(result_df, on=['date', 'district', 'category'], suffixes=('', '_avg'))

# Rename the 'usdprice_avg' column to 'average_usdprice'
s_df.rename(columns={'usdprice_avg': 'average_usdprice'}, inplace=True)

### filter dataset for only required columns

In [26]:
s_df = s_df.drop(['commodity', 'usdprice'], axis = 1)
s_df = s_df.drop_duplicates()
# 3 months before IPC ratings started
s_df = s_df[s_df['date'] >= '2009-05-01']

In [27]:
s_df

,date,district,ipc,category,average_usdprice
29,2009-05-01,Bor,NaN,cereals and tubers,1.873100
32,2009-05-01,Bor,NaN,pulses and nuts,4.508400
34,2009-06-01,Bor,NaN,cereals and tubers,1.658433
37,2009-06-01,Bor,NaN,pulses and nuts,3.135800
39,2009-07-01,Bor,2.0,cereals and tubers,1.720167
...,...,...,...,...,...
38705,2020-02-01,Malakal,4.0,"meat, fish and eggs",83.648700
38707,2020-02-01,Malakal,4.0,miscellaneous food,2.788300
38709,2020-02-01,Malakal,4.0,non-food,3.485350
38713,2020-02-01,Malakal,4.0,oil and fats,6.196200


# Correlation Work

### WIP
fix correlation with timeseries. Check if when removing null ipc values after rolling mean makes a difference

In [54]:
import numpy as np

In [63]:
# Create an empty dictionary to store correlations per category
correlations_per_category_district = {}

# Iterate through each unique category
for category in s_df['category'].unique():
    # Filter the DataFrame for the current category
    category_df = s_df[s_df['category'] == category]
    
    # Sort the data by date within each group
    category_df = category_df.sort_values(by=['district', 'date'])
    
    # Calculate the rolling average of 'usdprice' per 'district' for the current category
    category_df['rolling_usdprice'] = category_df.groupby(['district'])['average_usdprice'].rolling(window=3).mean().values
    category_df = category_df.dropna(subset=['ipc'])

# Iterate through each unique district within the current category
    for district in category_df['district'].unique():
        # Filter the data for the current district
        district_df = category_df[category_df['district'] == district]
        
        # Calculate the correlation between 'ipc' and 'rolling_usdprice' for the current district
        correlation = district_df['ipc'].corr(district_df['rolling_usdprice'])
        
        # Store the correlation in the dictionary with the category and district as the key
        key = (category, district)
        correlations_per_category_district[key] = correlation

# calcuate averages
average_dic = {}
for category in s_df['category'].unique():
    values = [value for key, value in correlations_per_category_district.items() if key[0] == category]
    average = np.nanmean(values)
    average_dic[category] = average

# Print the average correlations for each category over the districts
for category, correlation in average_dic.items():
    print(f"Correlation between 'category' **'{category}'** and 'ipc' with 3-month rolling average of 'usdprice':", correlation)
    print("")

Correlation between 'category' **'cereals and tubers'** and 'ipc' with 3-month rolling average of 'usdprice': 0.16187913428620299

Correlation between 'category' **'pulses and nuts'** and 'ipc' with 3-month rolling average of 'usdprice': -0.209452021565239

Correlation between 'category' **'non-food'** and 'ipc' with 3-month rolling average of 'usdprice': -0.28395347707441776

Correlation between 'category' **'oil and fats'** and 'ipc' with 3-month rolling average of 'usdprice': -0.313080390955397

Correlation between 'category' **'meat, fish and eggs'** and 'ipc' with 3-month rolling average of 'usdprice': -0.03280561033419541

Correlation between 'category' **'miscellaneous food'** and 'ipc' with 3-month rolling average of 'usdprice': -0.09762490125039858



c:\Users\pthie\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\pthie\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
